In [6]:
# VERSION 1
from google.colab import files
import pandas as pd

files.upload()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df1 = pd.read_csv('category_tree.csv')
df2s = []
start = pd.Timestamp('2015-09-01', tz='Asia/Ho_Chi_Minh')
end   = pd.Timestamp('2015-10-31 23:59:59', tz='Asia/Ho_Chi_Minh')
for i, chunk in enumerate(pd.read_csv('events.csv', chunksize=100000)):
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], unit='ms', utc=True, errors='coerce').dt.tz_convert('Asia/Ho_Chi_Minh')
    mask = (chunk['timestamp'] >= start) & (chunk['timestamp'] <= end)
    df2s.append(chunk.loc[mask])
    if i % 50 == 0:
        print(f"Processed {i*100000:,} rows...")
df2 = pd.concat(df2s)
df2.to_csv('events_cleaned.csv', index=False)
files.download('events_cleaned.csv')

for name, df in zip(['category_tree', 'events'], [df1, df2]):
    print(f"\nBlank Checking for {name}")
    print((df.isna().mean()*100)[(df.isna().mean()*100)>0].sort_values(ascending=False))
print(df1[df1['parentid'].isna()])

def outliers_checking(df):
    report = {}
    for col in df.select_dtypes(include='number').columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5*iqr
        upper_bound = q3 + 1.5*iqr
        outliers = ((df[col]<lower_bound)|(df[col]>upper_bound)).mean()*100
        if outliers > 0.5:
            report[col] = round(outliers,2)
    return pd.Series(report, name='outlier_pct').sort_values(ascending=False)
print('\ncategory_tree: Outlier checking')
print(outliers_checking(df1))
print('\nevents: Outlier checking')
print(outliers_checking(df2))

df1.to_csv('category_tree_cleaned.csv', index=False)
files.download('category_tree_cleaned.csv')

Saving category_tree.csv to category_tree (5).csv
Saving events.csv to events (5).csv
Processed 0 rows...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Blank Checking for category_tree
parentid    1.497903
dtype: float64

Blank Checking for events
transactionid    99.208917
dtype: float64
      categoryid  parentid
5            231       NaN
589          791       NaN
861         1490       NaN
939          431       NaN
1111         755       NaN
1142         378       NaN
1208        1579       NaN
1211        1394       NaN
1322         659       NaN
1329        1057       NaN
1375         859       NaN
1392         803       NaN
1395         250       NaN
1476        1452       NaN
1484        1182       NaN
1490        1692       NaN
1614        1600       NaN
1616        1482       NaN
1622        1224       NaN
1623        1532       NaN
1624         395       NaN
1628         653       NaN
1629         140       NaN
1650        1698       NaN
1657         679       NaN

category_tree: Outlier checking
Series([], Name: outlier_pct, dtype: object)

events: Outlier checking
Series([], Name: outlier_pct, dtype: object)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>